# 2 对称量化
以0点为中心进行两个集合之间的映射。

<img src="./img/2_1_sym.png" alt="对称量化的数轴变化" width="600" />

对称量化过程中0点是不变的，动态范围是对称的。



In [47]:
import torch

In [48]:
# 定义一个tensor
x = torch.tensor([5.47,3.08,-7.59,0.00,-1.95,-4.57,10.80])
x


tensor([ 5.4700,  3.0800, -7.5900,  0.0000, -1.9500, -4.5700, 10.8000])

In [49]:
x.dtype

torch.float32

现在我们使用以下公式对这个tensor进行量化，需要注意的是由于对称量化以0点展开，因此对称量化通常用于权重。

<img src="./img/2_3_sym.png" alt="对称量化公式" width="600" />

clip是裁剪操作，用于解决异常值带来的误差。异常值的相关内容将在第三节非对称量化中进行展示。

In [50]:
 # 1. 分别计算原始集合和目标集合的最大值的绝对值
# 原始集合最大值
a = x.abs().max()
# 目标集合最大值（此处我们使用INT8作为目标集合，因此最大值的绝对值为127）
q_max_val = 127
print(f"原始集合的最大值为：{x_max_val}, 目标集合的最大值为：{q_max_val}")

原始集合的最大值为：10.800000190734863, 目标集合的最大值为：127


In [51]:
# 2. 计算缩放因子
s =  a / q_max_val
print(f"{a} / {q_max_val} = {s}, \n s : {s}({s.dtype})")

10.800000190734863 / 127 = 0.08503936976194382, 
 s : 0.08503936976194382(torch.float32)


In [52]:
# 对原始集合进行缩放
q = x / s
print(f"{q},{q.dtype}")

tensor([ 64.3231,  36.2185, -89.2528,   0.0000, -22.9306, -53.7398, 127.0000]),torch.float32


In [53]:
# 缩放后进行四舍五入，转成整数
q = torch.round(q)
print(f"{q},{q.dtype}")

tensor([ 64.,  36., -89.,   0., -23., -54., 127.]),torch.float32


In [ ]:
# 将q转换成整型
q = q.to(torch.int8)
print(f"{q}")

tensor([ 64,  36, -89,   0, -23, -54, 127], dtype=torch.int8)


量化前后的对比变化

<img src="./img/2_2_sym.png" alt="对称量化" width="600" />

In [55]:
# 3. 反量化，由于模型在前向传播过程中默认是以浮点数进行计算的（不考虑对激活进行量化的情况下），所以我们在计算的时候需要对权重进行反量化用于计算输出
x_hat = q * s
print(f"{x_hat},{x_hat.dtype}")

tensor([ 5.4425,  3.0614, -7.5685,  0.0000, -1.9559, -4.5921, 10.8000]),torch.float32


In [57]:
# 查看反量化带来的损失
err = x - x_hat
print(f"x={x}")
print(f"x_hat={x_hat}")
print(f"err={err}")

x=tensor([ 5.4700,  3.0800, -7.5900,  0.0000, -1.9500, -4.5700, 10.8000])
x_hat=tensor([ 5.4425,  3.0614, -7.5685,  0.0000, -1.9559, -4.5921, 10.8000])
err=tensor([ 0.0275,  0.0186, -0.0215,  0.0000,  0.0059,  0.0221,  0.0000])


原始数据->量化->反量化示意：

<img src="./img/2_4_sym.png" alt="" width="800" />